In [353]:
import pandas as pd
import statsmodels.formula.api as smf
from collections import defaultdict

In [354]:
data = pd.read_csv("output/describeNetwork.csv", dtype={'primary_naics': object})

In [355]:
naics = pd.read_csv("data/naics.csv", dtype={'2012 NAICS US Code': object})

In [356]:
naics = naics.drop(naics.columns[[0,3,4]], axis=1)

In [357]:
naics.columns = ["naics", "indName"]

In [358]:
# data["naics2dig"] = data['primary_naics'].str[:2]
# data["naics3dig"] = data['primary_naics'].str[:3]

In [359]:
# data = data.merge(naics, left_on=["naics2dig"], right_on="naics").merge(naics, left_on=["naics3dig"], right_on="naics", suffixes=("2", "3"))
# data = data.drop(["naics2dig", "naics3dig"], axis=1)

In [360]:
# this is why we couldn't use 2-digit naics codes - they don't all match up to the crosswalk
naics[naics['naics'].str.len()==2]

,naics,indName
1,11,"Agriculture, Forestry, Fishing and Hunting"
132,21,"Mining, Quarrying, and Oil and Gas Extraction"
180,22,Utilities
205,23,Construction
930,42,Wholesale Trade
1402,51,Information
1480,52,Finance and Insurance
1569,53,Real Estate and Rental and Leasing
1624,54,"Professional, Scientific, and Technical Services"
1718,55,Management of Companies and Enterprises


In [361]:
data["naics2"] = data['primary_naics'].str[:2]
data["naics3"] = data['primary_naics'].str[:3]

In [362]:
data = data.merge(naics, left_on=["naics3"], right_on="naics", how="left")

In [363]:
data = data.drop("naics", axis=1)

In [364]:
data.head()

,Unnamed: 0,Facility,Degrees,Clustering Coefficient,Betweeness,Closeness,Eigenvector,primary_naics,parent_company_name,Community,naics2,naics3,indName
0,0,IBM CORP,236,0.264443,0.060645,0.728713,0.056001,334111,IBM CORP,0,33,334,Computer and Electronic Product Manufacturing
1,1,IBM CORP,236,0.264443,0.060645,0.728713,0.056001,334413,IBM CORP,0,33,334,Computer and Electronic Product Manufacturing
2,2,EASTMAN KODAK CO EASTMAN BUSINESS PARK,228,0.284218,0.058787,0.717349,0.123391,325992,EASTMAN KODAK CO,0,32,325,Chemical Manufacturing
3,3,NORLITE LLC,225,0.243294,0.106510,0.720157,0.211257,327992,TRADEBE ENVIRONMENTAL SERVICES LLC,1,32,327,Nonmetallic Mineral Product Manufacturing
4,4,ALCOA INC,193,0.332038,0.051080,0.667877,0.054540,331313,ALCOA INC,4,33,331,Primary Metal Manufacturing


In [365]:
data.to_csv('output/facilitiesWithNaics.csv')

In [366]:
communities = defaultdict(dict)

In [371]:
for i in data['Community'].unique():
    commData = data[data['Community']==i]
    communities[i]['ungroupedData'] = commData
    industries = commData.groupby(['naics3', 'indName']).size().order(ascending=False)
    industries = industries.reset_index()
    industries = industries.rename(columns = {0:'count'})
    industries['percent'] = (industries['count'] / len(commData))*100
    communities[i]['groupedData'] = industries

/Users/afair/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: FutureWarning: order is deprecated, use sort_values(...)


In [376]:
for i in communities:
    communities[i]['groupedData'].to_csv('output/community{}.csv'.format(i))